In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pathlib import Path

raw_dir = Path("../data_raw")
out_dir = Path("../data")
out_dir.mkdir(exist_ok=True, parents=True)

# 1.1 Read the block group CSV to define zone_id and a human-readable name
bg_csv = raw_dir / "Census_Block_Groups_20251209.csv"
bg = pd.read_csv(bg_csv, dtype=str)

bg.columns


Index(['the_geom', 'OBJECTID', 'CTBLOCKGROUP', 'TRACT', 'BLOCKGROUP',
       'globalid', 'SHAPE__Length', 'SHAPE__Area'],
      dtype='object')

In [2]:
# Clean zone_id: remove commas and whitespace from CTBLOCKGROUP
bg["zone_id"] = (
    bg["CTBLOCKGROUP"]
    .str.replace(",", "", regex=False)
    .str.strip()
)

# Simple readable label: "Tract XXXX, BG Y"
bg["name"] = (
    "Tract " + bg["TRACT"].str.replace(",", "", regex=False).str.strip() +
    ", BG " + bg["BLOCKGROUP"].str.replace(",", "", regex=False).str.strip()
)

# For now, basic attributes; you will join real jobs/rent later
zone_attrs = pd.DataFrame({
    "zone_id": bg["zone_id"],
    "name": bg["name"],
    "jurisdiction": "n/a",          # placeholder
    "total_jobs": 0,                # placeholder; join real jobs later
    "cluster_jobs": 0,              # placeholder
    "median_rent": pd.NA            # placeholder
})

zone_attrs.to_csv(out_dir / "zone_attributes.csv", index=False)
zone_attrs.head()


,zone_id,name,jurisdiction,total_jobs,cluster_jobs,median_rent
0,83123,"Tract 8312, BG 3",n/a,0,0,<NA>
1,18022,"Tract 1802, BG 2",n/a,0,0,<NA>
2,149012,"Tract 14901, BG 2",n/a,0,0,<NA>
3,191032,"Tract 19103, BG 2",n/a,0,0,<NA>
4,94001,"Tract 9400, BG 1",n/a,0,0,<NA>


In [3]:
bg_geo_path = raw_dir / "Census_Block_Groups_20251209.geojson"

gdf_bg = gpd.read_file(bg_geo_path)

# Ensure we have the same zone_id as before (ctblockgroup, without commas)
gdf_bg["zone_id"] = (
    gdf_bg["ctblockgroup"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.strip()
)

# Reproject to WGS84 for lat/lon (if not already)
if gdf_bg.crs is None:
    # if GeoJSON is already EPSG:4326, you can skip this
    gdf_bg.set_crs(epsg=4326, inplace=True)
else:
    gdf_bg = gdf_bg.to_crs(epsg=4326)

gdf_bg["lon"] = gdf_bg.geometry.centroid.x
gdf_bg["lat"] = gdf_bg.geometry.centroid.y

centroids = gdf_bg[["zone_id", "lat", "lon"]].copy()
centroids.head()


/tmp/ipykernel_174240/2494978110.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_bg["lon"] = gdf_bg.geometry.centroid.x
/tmp/ipykernel_174240/2494978110.py:21: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_bg["lat"] = gdf_bg.geometry.centroid.y


,zone_id,lat,lon
0,83123,32.864994,-117.247960
1,18022,32.762204,-117.125072
2,149012,32.758218,-117.012085
3,191032,33.259510,-117.050995
4,94001,32.866891,-117.117817


In [4]:
def haversine_km(lat1, lon1, lat2, lon2):
    """
    Compute great-circle distance in kilometers between two points
    given in decimal degrees.
    """
    R = 6371.0  # Earth radius in km
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = np.sin(dlat / 2.0) ** 2 + \
        np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c


In [5]:
# Merge centroids with themselves (cartesian join)
orig = centroids.rename(columns={"zone_id": "origin", "lat": "olat", "lon": "olon"})
dest = centroids.rename(columns={"zone_id": "dest", "lat": "dlat", "lon": "dlon"})

# Cartesian product
od = orig.assign(key=1).merge(dest.assign(key=1), on="key").drop("key", axis=1)

# Compute distance in km
od["distance_km"] = haversine_km(od["olat"], od["olon"], od["dlat"], od["dlon"])

od.head()


,origin,olat,olon,dest,dlat,dlon,distance_km
0,83123,32.864994,-117.24796,83123,32.864994,-117.247960,0.000000
1,83123,32.864994,-117.24796,18022,32.762204,-117.125072,16.202593
2,83123,32.864994,-117.24796,149012,32.758218,-117.012085,25.037729
3,83123,32.864994,-117.24796,191032,33.259510,-117.050995,47.553463
4,83123,32.864994,-117.24796,94001,32.866891,-117.117817,12.156910


In [6]:
# Use the same zone_attrs as written to zone_attributes.csv
zone_attrs = pd.read_csv(out_dir / "zone_attributes.csv", dtype={"zone_id": str})

jobs = zone_attrs[["zone_id", "total_jobs", "cluster_jobs"]].copy()
jobs = jobs.rename(columns={"zone_id": "dest"})
jobs[["total_jobs", "cluster_jobs"]] = jobs[["total_jobs", "cluster_jobs"]].fillna(0)

# Join jobs onto OD table by destination
od = od.merge(jobs, on="dest", how="left")
od[["total_jobs", "cluster_jobs"]] = od[["total_jobs", "cluster_jobs"]].fillna(0)
od.head()


,origin,olat,olon,dest,dlat,dlon,distance_km,total_jobs,cluster_jobs
0,83123,32.864994,-117.24796,83123,32.864994,-117.247960,0.000000,0,0
1,83123,32.864994,-117.24796,18022,32.762204,-117.125072,16.202593,0,0
2,83123,32.864994,-117.24796,149012,32.758218,-117.012085,25.037729,0,0
3,83123,32.864994,-117.24796,191032,33.259510,-117.050995,47.553463,0,0
4,83123,32.864994,-117.24796,94001,32.866891,-117.117817,12.156910,0,0


In [7]:
distance_thresholds = {
    15: 5.0,   # min,km
    30: 10.0,  
    45: 18.0   
}


In [8]:
def compute_distance_accessibility(od_df, mode_name):
    """
    od_df: DataFrame with origin, dest, distance_km, total_jobs, cluster_jobs
    For now, mode_name is just a label ('transit' or 'auto');
    you can adjust thresholds or weights by mode later if desired.
    """
    records = []

    # For simplicity, use a single 'time_period' (e.g., 'AllDay')
    time_period = "AllDay"

    for origin, group in od_df.groupby("origin"):
        row = {
            "zone_id": origin,
            "time_period": time_period,
            "mode": mode_name
        }

        for t, max_km in distance_thresholds.items():
            mask = group["distance_km"] <= max_km
            jobs_t = group.loc[mask, "total_jobs"].sum()
            cluster_t = group.loc[mask, "cluster_jobs"].sum()

            row[f"jobs_{t}"] = jobs_t
            if t == 45:
                row["cluster_jobs_45"] = cluster_t

        records.append(row)

    return pd.DataFrame.from_records(records)


In [9]:
acc_transit = compute_distance_accessibility(od, "transit")
acc_auto = compute_distance_accessibility(od, "auto")

accessibility = pd.concat([acc_transit, acc_auto], ignore_index=True)
accessibility.head()


,zone_id,time_period,mode,jobs_15,jobs_30,jobs_45,cluster_jobs_45
0,10001,AllDay,transit,0,0,0,0
1,100011,AllDay,transit,0,0,0,0
2,100012,AllDay,transit,0,0,0,0
3,10002,AllDay,transit,0,0,0,0
4,10003,AllDay,transit,0,0,0,0


In [10]:
total_regional_jobs = zone_attrs["total_jobs"].fillna(0).sum()
if total_regional_jobs == 0:
    # Avoid division by zero; you will update this once real job data is joined
    total_regional_jobs = 1

accessibility["access_index_45"] = accessibility["jobs_45"] / total_regional_jobs

cols = [
    "zone_id", "time_period", "mode",
    "jobs_15", "jobs_30", "jobs_45",
    "cluster_jobs_45", "access_index_45"
]

accessibility[cols].to_csv(out_dir / "accessibility_summary.csv", index=False)
accessibility[cols].head()


,zone_id,time_period,mode,jobs_15,jobs_30,jobs_45,cluster_jobs_45,access_index_45
0,10001,AllDay,transit,0,0,0,0,0.0
1,100011,AllDay,transit,0,0,0,0,0.0
2,100012,AllDay,transit,0,0,0,0,0.0
3,10002,AllDay,transit,0,0,0,0,0.0
4,10003,AllDay,transit,0,0,0,0,0.0
